In [1]:
import numpy as np
from glob import glob
import shutil
import cv2
import pandas as pd
import os


In [ ]:
#用来把ENet生成的excel汇总在一起的
# for i in range(5):
#     df = pd.read_excel('Desity_pred' + str(i) + '.xlsx')
#     names = list(df['PicName'])
#     preds = list(df['Prediction'])

In [4]:
#随便写的用来验证TF那边读取图片是不是对的
pic = cv2.imread('pics1\\A2_2.5x-0003_0.png',0)
print(pic)

[[192 187 184 ... 175 173 173]
 [193 189 185 ... 174 176 174]
 [199 193 187 ... 174 175 175]
 ...
 [185 183 183 ... 177 182 189]
 [186 184 181 ... 181 184 185]
 [187 186 183 ... 183 182 183]]


In [23]:
def copy_to(sc_path, des_path):
    files = glob(sc_path + '*.*')
    for file in files:
        name = list(file.split('\\'))[-1]
        shutil.copyfile(file, des_path + name)

In [32]:
copy_to('Ti-6Al-4V set 2\\6s8v_2.5x\\', 'ori_pics2\\')

In [34]:
for i in range(7):
    for l in ['v', 'h']:
        copy_to('Ti-6Al-4V set 2\\3s' + str(i+1) + l + '\\', 'ori_pics2\\')

In [2]:
df = pd.read_excel('clean excel.xlsx')
df.head()

,Sample ID.,Scan Speed,Laser Power,Hatch Spacing (um),Energy Density (J/mm3),Part Density (%)
0,A1,1200,280.0,140.00,37.037037,99.9080
1,A2,1800,280.0,140.00,24.691358,96.2680
2,A3,1500,366.6,140.00,38.793651,99.6250
3,A4,1500,308.9,197.12,23.215789,92.7525
4,A5,600,280.0,140.00,74.074074,91.4560


In [3]:
folder_names = list(df['Sample ID.'])
energy = list(df['Energy Density (J/mm3)'])
density = list(df['Part Density (%)'])
print(density)

[99.908, 96.268, 99.625, 92.7525, 91.456, 99.952, 99.596, 78.888, 99.98, 99.744, 99.99, 99.97, 87.958, 99.836, 97.406, 99.698, 93.63, 97.318, 99.836, 99.698, 91.976, 99.982, 99.792, 99.956, 99.852, 92.47]


In [3]:
files = glob('ori_pics1\\*.*')
for file in files:
    name = list(file.split('\\'))[-1][:-4]
    pic = cv2.imread(file)
    pic = cv2.resize(pic, [256,256])
    pic0 = cv2.flip(pic, 0)
    pic1 = cv2.flip(pic, 1)
    pic2 = cv2.flip(pic, -1)
    cv2.imwrite('pics1\\' + name + '.png', pic)
    cv2.imwrite('pics1\\' + name + '_0.png', pic0)
    cv2.imwrite('pics1\\' + name + '_1.png', pic1)
    cv2.imwrite('pics1\\' + name + '_2.png', pic2)

In [6]:
files = glob('pics1\\*.*')
for file in files:
    name = list(file.split('\\'))[-1]
    folder = list(name.split('_'))[0]
    if folder == 'C14' or folder == 'C15':
        folder = 'C1'
    cv2.imwrite('GT_density\\'+name, np.round(density[folder_names.index(folder)]-70)/30*255)

In [4]:
for i in range(5):
    files = glob('point_folder\\group'+str(i)+'\\*.*')
    pic = np.zeros(1)
    for file in files:
        name = list(file.split('\\'))[-1][:-8]
        cv2.imwrite('point_folder\\group'+str(i)+'\\' + name + '.png', pic)
        os.remove(file)